<a href="https://colab.research.google.com/github/TAbdullah-T/T5-SAD/blob/main/Deep%20Learning/CNN_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network (CNN) using Keras
This notebook will guide you through the process of creating a CNN model using Keras. Follow the steps and fill in the code blocks as you progress.

# Dataset Card: Men vs. Women Classification

## Overview
The Men vs. Women Classification dataset contains images of men and women intended for binary image classification tasks. The goal is to classify images based on gender.

- **Dataset URL:** [Men vs. Women Classification Dataset](https://www.kaggle.com/datasets/saadpd/menwomen-classification)
- **Dataset Size:** ~845 MB
- **Classes:** 2 (Men, Women)
- **Image Format:** JPEG

## Structure

### Folders
The dataset is organized into two main folders:

- `traindata/`:
  - `traindata/`: Contains the training images.
    - `men/`: Contains images of men.
    - `women/`: Contains images of women.

- `testdata/`:
  - `testdata/`: Contains the testing images.
    - `men/`: Contains images of men.
    - `women/`: Contains images of women.

### Example Files
Here are some example file names you might find in the dataset:

- `traindata/traindata/men/000000899.jpg`
- `traindata/traindata/women/00000001.jpg`
- `testdata/testdata/men/00000504.jpg`
- `testdata/testdata/women/00000002.jpg`

### Image Specifications
- **Resolution:** Varies
- **Color:** RGB

## Usage
This dataset is ideal for practicing binary image classification using Convolutional Neural Networks (CNNs). It can be used to train a model to distinguish between images of men and women.

## Step 1: Import Required Libraries
Begin by importing the necessary libraries.

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d saadpd/menwomen-classification

Dataset URL: https://www.kaggle.com/datasets/saadpd/menwomen-classification
License(s): copyright-authors
menwomen-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/menwomen-classification.zip -d /content


Archive:  /content/menwomen-classification.zip
replace /content/testdata/testdata/men/00000001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf
import keras
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split




## Step 2: Load and Preprocess Data
Load your dataset and preprocess it. This may include resizing images, normalizing pixel values, and splitting the data into training and validation sets.

In [ ]:
def load_and_resize_image(img_Path):
  image = Image.open(img_Path).convert('RGB')
  image = image.resize((100,100))
  return np.array(image)


def load_images_with_labels(folder_path):

  labels = ['men', 'women']
  img = []
  labels_ = []
  for label in labels :
    folder_path_label = os.path.join(folder_path, label)
    for filename in os.listdir(folder_path_label):
      img_path = os.path.join(folder_path_label, filename)
      img.append(load_and_resize_image(img_path))
      labels_.append(label)

  img = np.array(img)
  labels_ = np.array(labels_)
  return img, labels_

X, y = load_images_with_labels('/content/Dataset/testdata')

In [ ]:
# Optionally, you can convert the labels to numerical values if needed
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y_encoded, test_size=0.2, shuffle= True, random_state=0)

In [ ]:
X_train.shape , y_train.shape, X_val.shape

((1066, 100, 100, 3), (1066,), (267, 100, 100, 3))

In [ ]:
print((y_train))

[1 1 0 ... 1 1 1]


## Step 3: Data Augmentation
To prevent overfitting, augment your data using various transformations like rotation, zoom, flip, etc.

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    zoom_range = 0.2,
)
datagen.fit(X_train)

## Step 4: Build the CNN Model
Define the architecture of your CNN model. Start with convolutional layers followed by pooling layers, and end with fully connected layers.

In [ ]:

model = Sequential([

        Conv2D(32, (3,3), activation='relu', padding = 'same', input_shape=(100,100,3),
              kernel_regularizer=l1_l2(l1=0.01, l2=0.01),),

        BatchNormalization(),
        Dropout(0.5),
        Conv2D(32, (3,3), activation='relu', padding='same',kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        BatchNormalization(),
        Dropout(0.5),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu', padding='same',kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Conv2D(64, (3,3),activation='relu', padding='same',kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        BatchNormalization(),
        Dropout(0.5),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(128, activation='relu',kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        BatchNormalization(),
        Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Step 5: Compile the Model
Compile your model by specifying the optimizer, loss function, and evaluation metrics.

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Step 6: Train the Model
Train your model using the training data and validate it using the validation data.

In [ ]:
history = model.fit(datagen.flow(X_train,y_train, batch_size=64),
                    batch_size= 1000,
                    epochs = 10,
                    validation_data = (X_val,y_val),
                    verbose=1)

## Step 7: Evaluate the Model
Evaluate the performance of your model using the validation set.

In [ ]:
def load_resize_image(file_path):
  image = Image.open(file_path).convert('RGB')
  image = image.resize((100,100))
  return np.array(image )

def load_img_label(folder_path):
  labels = ['men','women']
  img = []
  labels_ = []
  for label in labels :
    label_path = os.path.join(folder_path, label)
    for filename in os.listdir(label_path) :
      img_path = os.path.join(label_path, filename)
      img.append(load_resize_image(img_path))
      labels_.append(label)

  return np.array(img) , np.array(labels_)

X_test, y_test = load_img_label('/content/Dataset/testdata')

In [ ]:
y_enc = label_encoder.transform(y_test)

In [ ]:
model.evaluate(X_test,y_enc)

42/42 ━━━━━━━━━━━━━━━━━━━━ 21s 493ms/step - accuracy: 0.6583 - loss: 41.3305


[79.00054168701172, 0.3150787651538849]

## Step 8: Save the Model
Finally, save your trained model for future use.